In [10]:
with open("gh_access_token", "r") as file:
    access_token = file.read().strip()

import requests
import json
import csv
import pandas as pd


csv_path = 'repos_with_datetime.csv'
df = pd.read_csv(csv_path)

In [28]:
query = """
query($q: String!, $cursor: String) {
  rateLimit {
    remaining
    cost
    used
  }
  search(query:$q, type: ISSUE, first: 100, after:$cursor) {
		pageInfo {
      hasNextPage
      endCursor
    }
    nodes {
      ... on Issue {
        
        title
        bodyHTML
        url
        activeLockReason
        
        
        labels (first:100) {
          nodes {
            name
          }
        }
        
      }
    }
  }
}
"""

In [26]:
with open("issues.csv", "w") as file:
  writer = csv.writer(file, lineterminator="\n")
  
  row = ["repoName", "title", "bodyHtml", "url", "lockReason", "labels"]
  writer.writerow(row)

In [27]:
url = "https://api.github.com/graphql"
headers = {"Authorization": f"Bearer {access_token}"}

def search_issues(nameWithOwner):
  count = 0
  q = f"repo:{nameWithOwner} is:issue is:closed \"datetime\" AND \"bug\""
  cursor = None
  while (True):
    json = {"query": query, "variables": {"q": q, "cursor": cursor}}
    response = requests.post(url, json=json, headers=headers).json()["data"]

    rateLimit = response["rateLimit"]
    hasNextPage = response["search"]["pageInfo"]["hasNextPage"]
    cursor = response["search"]["pageInfo"]["endCursor"]

    issues = response["search"]["nodes"]


    with open("issues.csv", "a") as file:
      writer = csv.writer(file, lineterminator="\n")

      for issue in issues:
        labels = []

        for l in issue["labels"]["nodes"]:
          labels.append(l["name"])


        row = [nameWithOwner, issue["title"], "<html redacted>", issue["url"], # issue["bodyHTML"]
               issue["activeLockReason"], labels
        ]
        writer.writerow(row)

    if (count % 1 == 0):
      print(f"Requests: {count}, endCursor: {cursor}, remaining: {rateLimit['remaining']}")

    count += 1

    if (not hasNextPage):
      print("done")
      break

for index, row in df.iterrows():
  nameWithOwner = row["nameWithOwner"]
  search_issues(nameWithOwner)


Requests: 0, endCursor: Y3Vyc29yOjE=, remaining: 4948
done
Requests: 0, endCursor: Y3Vyc29yOjE=, remaining: 4947
done
Requests: 0, endCursor: None, remaining: 4946
done
Requests: 0, endCursor: Y3Vyc29yOjU=, remaining: 4945
Requests: 1, endCursor: Y3Vyc29yOjEw, remaining: 4944
Requests: 2, endCursor: Y3Vyc29yOjEy, remaining: 4943
done
Requests: 0, endCursor: Y3Vyc29yOjU=, remaining: 4942
Requests: 1, endCursor: Y3Vyc29yOjEw, remaining: 4941
Requests: 2, endCursor: Y3Vyc29yOjE1, remaining: 4940
done
Requests: 0, endCursor: Y3Vyc29yOjU=, remaining: 4939
Requests: 1, endCursor: Y3Vyc29yOjEw, remaining: 4938
Requests: 2, endCursor: Y3Vyc29yOjEx, remaining: 4937
done


In [9]:
df

,name,url,description,stars,primaryLanguage,updatedAt,createdAt,issuesCount,forkCount,watchersCount,discussionsCount,contains_import
0,Python,https://github.com/TheAlgorithms/Python,All Algorithms implemented in Python,181296,Python,2024-06-11T18:44:53Z,2016-07-16T09:44:01Z,1451,43888,5929,58,1
1,AutoGPT,https://github.com/Significant-Gravitas/AutoGPT,AutoGPT is the vision of accessible AI for eve...,162950,Python,2024-06-11T17:29:33Z,2023-03-16T09:21:07Z,2225,43131,1555,1084,1
2,Python-100-Days,https://github.com/jackfrued/Python-100-Days,Python - 100天从新手到大师,151569,Python,2024-06-11T16:49:46Z,2018-03-01T16:05:52Z,636,51509,6136,0,1
3,stable-diffusion-webui,https://github.com/AUTOMATIC1111/stable-diffus...,Stable Diffusion web UI,133448,Python,2024-06-11T18:48:54Z,2022-08-22T14:05:26Z,7453,25587,1039,4783,1
4,youtube-dl,https://github.com/ytdl-org/youtube-dl,Command-line program to download videos from Y...,129343,Python,2024-06-11T18:42:14Z,2010-10-31T14:35:07Z,26517,9723,2198,0,1
5,transformers,https://github.com/huggingface/transformers,🤗 Transformers: State-of-the-art Machine Learn...,127430,Python,2024-06-11T18:39:22Z,2018-10-29T13:56:00Z,14975,25257,1101,0,1
